In [250]:
import numpy as np

In [251]:
data = open('data/day20.txt', 'r').read()

In [252]:
example = """broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a"""

In [253]:
example2 = """broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output"""

In [254]:
def parse_configuration(data):
    """
    Parse the data into the necessary structures
    """
    configuration = {}
    on_status_tracker = {}
    high_pulse_tracker = {('button', 'broadcaster'): False}
    for line in data.split('\n'):
        source_module, dest_modules = line.split(' -> ')
        dest_modules = dest_modules.split(', ')
        if '%' in source_module:
            source_module = source_module.replace('%', '')
            module_type = 'flip-flop'
            on_status_tracker[source_module] = False
        elif '&' in source_module:
            source_module = source_module.replace('&', '')
            module_type = 'conjunction'
        else:
            module_type = source_module
        configuration[source_module] = (module_type, dest_modules)
        
        for dest_module in dest_modules:
            high_pulse_tracker[(source_module, dest_module)] = False
    
    return configuration, on_status_tracker, high_pulse_tracker

In [255]:
def update_module(on_status_tracker, high_pulse_tracker,
                  current_module, module_type, dest_modules,
                  is_high_pulse):
    """
    Updates module and sends a bool indicating whether the pulse is a high pulse or not
    """
    # Update module and identify the return pulse type
    if module_type == 'flip-flop' and not is_high_pulse:
        # If it was off, turn it on and send high pulse, else turn it off and send low pulse
        on_status_tracker[current_module] = not on_status_tracker[current_module]
        return_is_high_pulse = on_status_tracker[current_module]
    elif module_type == 'conjunction':
        # If connected modules all last sent high pulses, send low pulse, else send high pulse
        return_is_high_pulse = not all([is_high_pulse 
                                        for ((source_module, dest_module), is_high_pulse) 
                                        in high_pulse_tracker.items()
                                        if dest_module == current_module])
    else:
        # Return the input pulse type
        return_is_high_pulse = is_high_pulse
    
    # Update the high pulse tracker
    for dest_module in dest_modules:
        high_pulse_tracker[(current_module, dest_module)] = return_is_high_pulse    
    
    return on_status_tracker, high_pulse_tracker
        

In [256]:
def handle_button_push(configuration, on_status_tracker, high_pulse_tracker, pulse_counts):
    """
    Push the button and handle the subsequent module updates, returning the count of high/low pulses
    """    
    # Push the button
    module_queue = [('button', 'broadcaster')]
    
    while module_queue:
        (source_module, current_module) = module_queue.pop(0)
        is_high_pulse = high_pulse_tracker[(source_module, current_module)]
        pulse_counts['high' if is_high_pulse else 'low'] += 1
        if current_module not in configuration:
            continue
        else:
            module_type, dest_modules = configuration[current_module]

        if not (module_type == 'flip-flop' and is_high_pulse):
            on_status_tracker, high_pulse_tracker = update_module(on_status_tracker,
                                                                  high_pulse_tracker,
                                                                  current_module,
                                                                  module_type,
                                                                  dest_modules,
                                                                  is_high_pulse)
            for dest_module in dest_modules:
                module_queue += [(current_module, dest_module)]
    
    return on_status_tracker, high_pulse_tracker, pulse_counts

In [264]:
def find_cycle(configuration, on_status_tracker, high_pulse_tracker, pulse_counts, num_button_pushes):
    """
    Find the # of button pushes and pulses needed for the flip-flop modules to reset
    """
    num_cycle_button_pushes = 0
    while num_cycle_button_pushes == 0 or any(list(on_status_tracker.values())):
        num_cycle_button_pushes += 1
        on_status_tracker, high_pulse_tracker, pulse_counts = handle_button_push(configuration,
                                                                             on_status_tracker,
                                                                             high_pulse_tracker,
                                                                             pulse_counts)
        if num_cycle_button_pushes == num_button_pushes:
            break
    
    return num_cycle_button_pushes, pulse_counts    

In [302]:
def part1(data, num_button_pushes):
    configuration, on_status_tracker, high_pulse_tracker = parse_configuration(data)   
    num_cycle_button_pushes, cycle_pulse_counts = find_cycle(configuration,
                                                             on_status_tracker,
                                                             high_pulse_tracker,
                                                             {'high': 0, 'low': 0},
                                                             num_button_pushes)
    
    num_cycles = num_button_pushes / num_cycle_button_pushes
    total_pulse_counts = np.array(list(cycle_pulse_counts.values())) * num_cycles
    
    return np.int64(np.prod(total_pulse_counts))
part1(data, 1000)

739960225

In [373]:
def find_needed_high_pulses(configuration, on_status_tracker, high_pulse_tracker,
                            need_high_pulse, button_push_index):
    """
    Push the button and handle the subsequent updates, flagging when we find the needed high pulses
    """    
    # Push the button
    module_queue = [('button', 'broadcaster')]
    
    while module_queue:
        (source_module, current_module) = module_queue.pop(0)
        is_high_pulse = high_pulse_tracker[(source_module, current_module)]
        # Check if we've found a high pulse for any of the needed modules
        if (source_module, current_module) in need_high_pulse:
            if is_high_pulse and need_high_pulse[(source_module, current_module)] is None:
                need_high_pulse[(source_module, current_module)] = button_push_index
        
        if current_module not in configuration:
            continue
        else:
            module_type, dest_modules = configuration[current_module]

        if not (module_type == 'flip-flop' and is_high_pulse):
            on_status_tracker, high_pulse_tracker = update_module(on_status_tracker,
                                                                  high_pulse_tracker,
                                                                  current_module,
                                                                  module_type,
                                                                  dest_modules,
                                                                  is_high_pulse)
            for dest_module in dest_modules:
                module_queue += [(current_module, dest_module)]
    
    return on_status_tracker, high_pulse_tracker, need_high_pulse

In [375]:
def part2(data):
    """
    This is fairly hard-coded but essentially rx is only connected to one conjunction module gf
    That conjunction module is connected to 4 other conjunction modules
    
    So if we want to have gf produce a low pulse, 
    all 4 of the connected conjunctions need to send a high pulse
    
    Thus, we just need to find the lcm of when the connected conjunctions first send a high pulse
    
    The key is that we need to check for this within a single button push, not following a button push
    """
    configuration, on_status_tracker, high_pulse_tracker = parse_configuration(data)   
    need_high_pulse = {(source, dest): None for (source, dest) in high_pulse_tracker if dest == 'gf'}
    
    button_push_index = 0
    while any([val is None for val in need_high_pulse.values()]):
        button_push_index += 1
        on_status_tracker, high_pulse_tracker, need_high_pulse = find_needed_high_pulses(configuration,
                                                                                         on_status_tracker,
                                                                                         high_pulse_tracker,
                                                                                         need_high_pulse,
                                                                                         button_push_index)
    return math.lcm(*need_high_pulse.values())
part2(data)

231897990075517